In [28]:
!pip install selenium webdriver-manager pandas

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException


import pandas as pd
import numpy as np
import logging
import os
import time
import datetime


# Configuração de logs
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")

In [3]:
def test_driver_para_chrome():
    """
    Configura o Selenium WebDriver com Chrome e baixa automaticamente o ChromeDriver.
    """
    
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager


    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Executa sem interface gráfica
    options.add_argument("--no-sandbox")  # Desativa o sandbox
    options.add_argument("--disable-dev-shm-usage")  # Resolve problemas de memória compartilhada
    options.add_argument("--disable-gpu")  # Necessário para ambientes headless em alguns sistemas

    # Configura serviço do ChromeDriver
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

# Testar a configuração do driver
try:
    driver = test_driver_para_chrome()
    print("ChromeDriver configurado com sucesso.")
    driver.quit()
except Exception as e:
    print(f"Erro ao configurar o ChromeDriver: {e}")


2024-12-25 19:36:18,928 - ====== WebDriver manager ======
2024-12-25 19:36:20,268 - Get LATEST chromedriver version for google-chrome
2024-12-25 19:36:20,317 - Get LATEST chromedriver version for google-chrome
2024-12-25 19:36:20,353 - Driver [C:\Users\flavio.lopes\.wdm\drivers\chromedriver\win64\131.0.6778.204\chromedriver-win32/chromedriver.exe] found in cache


ChromeDriver configurado com sucesso.


In [7]:
def configurar_driver_para_chrome(driver_path=None):
    """
    Configura o Selenium WebDriver com Chrome e verifica se o ChromeDriver já está baixado.
    Caso não esteja, faz o download.

    Args:
        driver_path (str): Caminho para o ChromeDriver já baixado, se disponível.

    Returns:
        WebDriver: Instância do WebDriver configurada.
    """
    # Opções para otimizar o Chrome WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Executa o navegador sem interface gráfica, ideal para ambientes de servidor ou testes automatizados.
    options.add_argument("--no-sandbox")  # Desativa o sandbox do Chrome, necessário para evitar problemas de permissões em alguns sistemas Linux.
    options.add_argument("--disable-dev-shm-usage")  # Redireciona o uso de memória compartilhada para o disco, resolve problemas em sistemas com memória limitada.
    options.add_argument("--disable-gpu")  # Desativa a renderização por GPU. Essencial em modo headless para evitar problemas em alguns sistemas.
    options.add_argument("--disable-extensions")  # Desativa todas as extensões do navegador, melhorando o desempenho e evitando interferências.
    options.add_argument("--disable-notifications")  # Bloqueia notificações push, evitando interrupções durante a automação.
    options.add_argument("--disable-application-cache")  # Desativa o cache de aplicativos para garantir que os dados sejam sempre carregados do servidor.
    options.add_argument("--disk-cache-size=0")  # Define o tamanho do cache no disco como zero, reduzindo o uso de armazenamento local.
    options.add_argument("--disable-blink-features=AutomationControlled")  # Remove sinais de automação do navegador para evitar detecção como Selenium por parte dos sites.


    # Define um user agente de um navegador real
    # Para obter mais user agents consulte https://www.useragentstring.com/
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    options.add_argument(f"user-agent={user_agent}")
    
    if driver_path and os.path.exists(driver_path):
        logging.info(f"Usando ChromeDriver existente em: {driver_path}")
        service = Service(driver_path)
    else:
        logging.info("ChromeDriver não encontrado no caminho especificado. Baixando novo driver...")
        service = Service(ChromeDriverManager().install())

    return webdriver.Chrome(service=service, options=options)

In [5]:
def is_valid_date(date_string):
    try:
        # Tenta converter a string para datetime no Pandas
        pd.to_datetime(date_string, format="%Y-%m-%d", errors="raise")
        return True
    except ValueError:
        return False

def is_in_iframe(driver, iframe_id):
    current_iframe_id = driver.execute_script(
        "return window.frameElement ? window.frameElement.id : null;"
    )

    return (current_iframe_id==iframe_id)

def select_iframe(driver, iframe_id):
    # Voltar ao contexto principal (iframe original)
    driver.switch_to.default_content()

    if not is_in_iframe(driver, iframe_id):
        iframe = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, iframe_id))
        )
        driver.switch_to.frame(iframe)
    
def extrair_dados_ibovespa(semestre, driver):
    """
    Extrai os dados da tabela do Ibovespa para o semestre especificado.

    Args:
        semestre (str): '1' para 1º semestre ou '2' para 2º semestre.
        driver (webdriver): Instância do Selenium WebDriver.

    Returns:
        pd.DataFrame: Dados extraídos no formato original.
    """
    try:

        # Mudar para o iframe onde os dados estão
        if not is_in_iframe(driver,"bvmf_iframe"):
            select_iframe(driver, "bvmf_iframe")

        # Seleciona o semestre no dropdown
        logging.info(f"Selecionando semestre {semestre}...")
        select = Select(driver.find_element(By.ID, "semester"))
        select.select_by_value(semestre)
        #time.sleep(3)

        # Aguarda a tabela carregar
        tabela_xpath = "/html/body/app-root/app-daily-evolution/div/div/div[1]/form/div[2]/div/table"
        tabela = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, tabela_xpath))
        )

        # Extrai os dados da tabela
        linhas = tabela.find_elements(By.TAG_NAME, "tr")
        dados = []
        for linha in linhas:
            colunas = linha.find_elements(By.TAG_NAME, "td")
            if colunas:
                dados.append([coluna.text for coluna in colunas])

        # Voltar ao contexto principal (iframe original)
        driver.switch_to.default_content()
        
        # Retorna os dados como DataFrame
        if semestre=='1':
            colunas = ["Dia", "Jan", "Fev", "Mar", "Abr", "Mai", "Jun"]
        if semestre=='2':
            colunas = ["Dia", "Jul", "Ago", "Set", "Out", "Nov", "Dez"]

        return pd.DataFrame(dados, columns=colunas)

    except Exception as e:
        logging.error(f"Erro ao extrair os dados do semestre {semestre}: {e}")
        return pd.DataFrame()

def transformar_dados(ano, dados):
    """
    Converte a tabela de dias e meses em formato de linhas únicas, com colunas Data e Valor.

    Args:
        dados (pd.DataFrame): DataFrame com colunas representando os meses e os dias como índice.

    Returns:
        pd.DataFrame: DataFrame transformado com colunas Data e Valor.
    """
    try:
        registros = []
        meses_map = {
            "Jan": "01", "Fev": "02", "Mar": "03", "Abr": "04",
            "Mai": "05", "Jun": "06", "Jul": "07", "Ago": "08",
            "Set": "09", "Out": "10", "Nov": "11", "Dez": "12"
        }

        for _, row in dados.iterrows():
            dia = row["Dia"]
            for mes, valor in row.items():
                # cabeçalhos e rodapés ['Dia', 'MÁXIMO', 'MÍNIMO']
                ignorar=['Dia', 'MÁXIMO', 'MÍNIMO']
                if mes not in ignorar and dia.isdigit(): # Ignora colunas vazias, de cabeçalho, rodapé e meses não numéricos
                    mes_num = meses_map.get(mes, None)
                    if mes_num:
                        data = f"{ano}-{mes_num.zfill(2)}-{str(dia).zfill(2)}"
                        valor=str(valor)
                        if valor.strip() !='':
                            valor=valor.replace('.','')
                            valor=valor.replace(',','.')
                            #valor=float(valor)
                        if is_valid_date(data):    
                            registros.append({"date": data, "value": valor})

        df_registros=pd.DataFrame(registros)
                
        # Ordenar o DataFrame pela coluna 'Data'
        df_registros = df_registros.sort_values(by="date")

        print('Estrutura do Dataframe')
        print(df_registros.info())

        # Preencher os valores ausentes: primeiro com forward fill, depois com backward fill
        # Isso é necessário caso o primeiro valor esteja em branco
        df_registros["value"] = df_registros["value"].replace('', np.nan)
        df_registros["value"] = df_registros["value"].ffill()
        df_registros["value"] = df_registros["value"].bfill()

        # Converte a coluna para numero decimal
        df_registros["value"] = pd.to_numeric(df_registros["value"], errors="coerce").round(2)

               
        return df_registros
    
    except Exception as e:
        logging.error(f"Erro ao transformar os dados: {e}")
        raise(e)
        return pd.DataFrame()

In [10]:
def obter_driver():
    # Caminho opcional para o ChromeDriver
    caminho_chromedriver = "./chromedriver"  # Substitua pelo caminho correto, se necessário

    try:
        # Configura o driver
        driver = configurar_driver_para_chrome(driver_path=caminho_chromedriver)
        return driver
    except Exception as e:
        logging.error(f"Erro ao obter o conteudo da pagina: {e}")
        return None

def obter_conteudo_pagina_ibov(driver):
    try:
        # Acessa o site
        url = "https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-estatisticas-historicas.htm"
        logging.info("Acessando o site da B3...")
        driver.get(url)
    except Exception as e:
        logging.error(f"Erro ao obter o conteudo da pagina: {e}")

def selecionar_ano(driver, ano):
    """
    Seleciona um ano no dropdown selectYear.

    Args:
        driver (webdriver): Instância do Selenium WebDriver.
        ano (str): Ano a ser selecionado no dropdown.

    Returns:
        bool: True se o ano foi selecionado com sucesso, False caso contrário.
    """
    try:

        if not is_in_iframe(driver, "bvmf_iframe"):
            # Mudar para o iframe onde os dados estão
            select_iframe(driver, "bvmf_iframe")
            
        # Localizar o elemento <select> pelo ID
        select_element = driver.find_element(By.ID, "selectYear")
        
        # Criar o objeto Select
        select = Select(select_element)
        
        # Selecionar o ano pelo valor
        select.select_by_value(ano)
        print(f"Ano {ano} selecionado com sucesso.")
        return True
    except NoSuchElementException:
        print(f"Ano {ano} não encontrado no dropdown selectYear.")
        return False
    except Exception as e:
        print(f"Erro ao selecionar o ano {ano}: {e}")
        return False

def get_all_years(driver):

    if not is_in_iframe(driver, "bvmf_iframe"):
        select_iframe(driver, "bvmf_iframe")

    # Localizar o elemento <select> pelo ID
    select_element = driver.find_element(By.ID, "selectYear")

    # Criar um objeto Select para interagir com o <select>
    select = Select(select_element)

    # Obter todos os valores disponíveis nas opções
    anos_disponiveis = [option.get_attribute("value") for option in select.options]

    # Converter para inteiros e ordenar
    anos_ordenados = sorted(anos_disponiveis, key=int)

    return anos_ordenados

def obter_ibov_b3(driver=None, ano=None, pagina_ibov_selecionada=False):
    is_external_driver=(driver!=None)

    # Configura o driver
    if driver==None:
        driver = obter_driver()
    
    try:
        # Obtem o conteudo do site
        if not pagina_ibov_selecionada:
            obter_conteudo_pagina_ibov(driver)
        
        # Selecionar Ano, se especificado
        if ano != None:
            logging.info(f"Selecionando o ano {ano}...")
            selecionar_ano(driver, ano)
        else:
            ano=datetime.now().year 

        # Extrai os dados dos dois semestres
        dados_1_semestre = extrair_dados_ibovespa("1", driver)
        print('Dados do Primeiro Semestre')
        print(dados_1_semestre.head())
        dados_transformados_semestre_1=transformar_dados(ano, dados_1_semestre)
        #dados_transformados_semestre_1.to_csv('dados_transformados_semestre_1.csv', index=False)

        dados_2_semestre = extrair_dados_ibovespa("2", driver)
        print('Dados do Segundo Semestre')
        print(dados_2_semestre.head())
        dados_transformados_semestre_2=transformar_dados(ano, dados_2_semestre)
        #dados_transformados_semestre_2.to_csv('dados_transformados_semestre_2.csv', index=False)
      
        # Combina os dados dos dois semestres
        dados_completos = pd.concat([dados_transformados_semestre_1, dados_transformados_semestre_2], ignore_index=True)
        
        if os.path.exists('ibovespa_diario.csv'):
            dados_existente=pd.read_csv('ibovespa_diario.csv')
            dados_completos=pd.concat([dados_completos, dados_existente], ignore_index=True)
            # Ordenar o DataFrame pela coluna 'Data'
            dados_completos = dados_completos.sort_values(by="date")
            dados_completos = dados_completos.drop_duplicates(subset="date")
   
        # Obter a data atual e formatá-la no formato yyyy-MM-dd
        data_atual = datetime.datetime.now().strftime("%Y-%m-%d")
        logging.info(f"Data atual formatada: {data_atual}")

        # Filtrar as datas até a data atual
        dados_completos["date"] = pd.to_datetime(dados_completos["date"])  # Converter para datetime
        dados_completos = dados_completos[dados_completos["date"] <= pd.to_datetime(data_atual)]

        # Exibe os dados
        logging.info("Dados transformados com sucesso. Exibindo os dados:")
        print(dados_completos.head())

        # Salva os dados em CSV
        dados_completos.to_csv("ibovespa_diario.csv", index=False)
        logging.info("Dados salvos em 'ibovespa_diario.csv'.")

    except Exception as e:
        logging.error(f"Erro durante a execução: {e}")

    finally:
        if not is_external_driver:
            # Fecha o navegador
            driver.quit()
            logging.info("Driver encerrado.")

try:
    ano_selecionado='2024'
    driver = obter_driver()

    if not os.path.exists('ibovespa_diario.csv'):
        obter_conteudo_pagina_ibov(driver)
        anos_disponiveis=get_all_years(driver)
        print(anos_disponiveis)
                
        for ano in anos_disponiveis:
            obter_ibov_b3(driver=driver, ano=ano, pagina_ibov_selecionada=True)
    else:
        obter_ibov_b3(driver=driver, ano=ano_selecionado, pagina_ibov_selecionada=False)
    
except Exception as e:
    logging.error(f"Erro durante a execução: {e}")
finally:
    driver.quit()
    logging.info("Driver encerrado.")   

2024-12-26 13:50:31,276 - ChromeDriver não encontrado no caminho especificado. Baixando novo driver...
2024-12-26 13:50:31,278 - ====== WebDriver manager ======
2024-12-26 13:50:32,910 - Get LATEST chromedriver version for google-chrome
2024-12-26 13:50:32,965 - Get LATEST chromedriver version for google-chrome
2024-12-26 13:50:33,012 - Driver [C:\Users\flavio.lopes\.wdm\drivers\chromedriver\win64\131.0.6778.204\chromedriver-win32/chromedriver.exe] found in cache
2024-12-26 13:50:38,340 - Acessando o site da B3...
2024-12-26 13:50:41,674 - Selecionando o ano 2024...
2024-12-26 13:50:41,819 - Selecionando semestre 1...


Ano 2024 selecionado com sucesso.


2024-12-26 13:50:44,128 - Selecionando semestre 2...


Dados do Primeiro Semestre
  Dia         Jan         Fev         Mar         Abr         Mai         Jun
0   1              128.481,02  129.180,37  126.990,45                        
1   2  132.696,63  127.182,25              127.548,52  127.122,25            
2   3  132.833,95                          127.318,39  128.508,67  122.031,58
3   4  131.225,91              128.340,54  127.427,53              121.802,06
4   5  132.022,92  127.593,49  128.098,11  126.795,41              121.407,33
Estrutura do Dataframe
<class 'pandas.core.frame.DataFrame'>
Index: 182 entries, 0 to 178
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    182 non-null    object
 1   value   182 non-null    object
dtypes: object(2)
memory usage: 4.3+ KB
None


2024-12-26 13:50:46,408 - Data atual formatada: 2024-12-26
2024-12-26 13:50:46,414 - Dados transformados com sucesso. Exibindo os dados:
2024-12-26 13:50:46,430 - Dados salvos em 'ibovespa_diario.csv'.


Dados do Segundo Semestre
  Dia         Jul         Ago         Set         Out         Nov         Dez
0   1  124.718,07  127.395,10              132.495,16  128.120,75            
1   2  124.787,08  125.854,09  134.906,07  133.514,94              125.235,54
2   3  125.661,89              134.353,48  131.671,51              126.139,20
3   4  126.163,98              136.110,73  131.791,55  130.514,79  126.087,02
4   5  126.267,05  125.269,54  136.502,49              130.660,75  127.857,58
Estrutura do Dataframe
<class 'pandas.core.frame.DataFrame'>
Index: 184 entries, 0 to 183
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    184 non-null    object
 1   value   184 non-null    object
dtypes: object(2)
memory usage: 4.3+ KB
None
          date     value
366 1998-01-01  10479.85
367 1998-01-02  10479.85
368 1998-01-03  10479.85
369 1998-01-04  10479.85
370 1998-01-05  10606.92


2024-12-26 13:50:48,607 - Driver encerrado.
